## Preamble

In [9]:
from scipy.special import comb
from itertools import product
import pandas as pd

## Functions

In [7]:
def hypergeo(k,n,K,N):
    return comb(n,k)*comb(N-n,K-k)/comb(N,K)

def hypergeo2events(k1,n1,k2,n2,K,N):
    return comb(n1,k1)*comb(n2,k2)*comb(N-n1-n2,K-k1-k2)/comb(N,K)

def hypergeoNevents(ks,ns,K,N):
    if len(ks) != len(ns):
        raise Exception("Input list sizes do not agree")
    result = comb(N-sum(ns),K-sum(ks))
    for i in range(0,len(ns)):
        k = ks[i]
        n = ns[i]
        result = result*comb(n,k)
    return result/comb(N,K)
        
def hypergeo_cumulative(k,n,K,N):
    result = 0
    for i in range(0,k+1):
        result += hypergeo(i,n,K,N)
    return result

def hypergeo2events_cumulative(k1,n1,k2,n2,K,N,intersection=True):
    result = 0
    i_s = range(0,k1+1)
    j_s = range(0,k2+1)
    for e in product(i_s, j_s):
        result += hypergeo2events(e[0],n1,e[1],n2,K,N)
    if intersection:
        for e in product(range(0,k1), range(k2+1,n2+1)):
            result += hypergeo2events(e[0],n1,e[1],n2,K,N)
        for e in product(range(k1+1,n1+1), range(0,k2)):
            result += hypergeo2events(e[0],n1,e[1],n2,K,N)
    return result

def hypergeoNevents_cumulative(ks,ns,K,N,intersection=True):
    result = 0
    inputs = []
    for k in ks:
        inputs.append(range(0,k+1))
    for e in product(*inputs):
        result += hypergeoNevents(list(e),ns,K,N)
    if intersection:
        for i in range(0,len(ks)):
            j=0
            inputs = []
            for k in ks:
                if i == j:
                    inputs.append(range(0,k))
                else:
                    inputs.append(range(k+1,ns[j]+1))
                j+=1
            for e in product(*inputs):
                result += hypergeoNevents(list(e),ns,K,N)
    return result

def distribute_k_over_n(k, l):
    result = []
    if k == 0:
        result.append(l)
    else:
        for i in range(0,len(l)):
            new_l = l.copy()
            new_l[i] += 1
            result.extend(distribute_k_over_n(k-1,new_l))
    return result

In [3]:
def trunc(s):
    return ((s*10000)//1)/10000

## Problem: Probabilities associated with A = drawing at least one of a card with a given CMC in a game and B = drawing enough lands to cast that card in the game

In [4]:
# number of cards in the deck
N = 99

# number of the given card in the deck
n1 = 1

# number of lands in the deck
n2 = 40

# initial number of cards drawn
K = 7

# number of the given card drawn
k1 = 1

# CMC of the card, i.e. number of lands drawn
k2 = 3

### P(A) over a number of draws

In [5]:
data = {f'Draw {i}' : [1-hypergeo_cumulative(k1-1,n1,K+i,N)] for i in range(0,11)}
pa_df = pd.DataFrame(data = data)
pa_df

,Draw 0,Draw 1,Draw 2,Draw 3,Draw 4,Draw 5,Draw 6,Draw 7,Draw 8,Draw 9,Draw 10
0,0.070707,0.080808,0.090909,0.10101,0.111111,0.121212,0.131313,0.141414,0.151515,0.161616,0.171717


### P(B) over a number of draws

In [6]:
data = {f'Draw {i}' : [1-hypergeo_cumulative(k2-1,n2,K+i,N)] for i in range(0,11)}
pb_df = pd.DataFrame(data = data)
pb_df

,Draw 0,Draw 1,Draw 2,Draw 3,Draw 4,Draw 5,Draw 6,Draw 7,Draw 8,Draw 9,Draw 10
0,0.593712,0.702056,0.787779,0.852684,0.900086,0.933662,0.956819,0.972411,0.982684,0.989316,0.993517


### P(A and B) over a number of draws

In [7]:
#data = {f'Draw {i}' : [1-hypergeo2events_cumulative(k1,n1,k2,n2,K+i,N)] for i in range(0,11)}
#pab_df = pd.DataFrame(data = data)
#pab_df

In [8]:
def prob(k, K, n1, n2, N, debug=False):    
    total = 0

    for k2 in range(k, K + 1):
        for k1 in range(1, n1 + 1):
            ks = [k1, k2]
            ns = [n1, n2]

            p = hypergeoNevents(ks,ns,K,N)
            
            total += p
            
            if debug:
                print((k1,k2), p, total)
        
    return total

In [9]:
data = {f'Draw {i}' : [prob(k2,K+i,n1,n2,N)] for i in range(0,11)}
pab_df = pd.DataFrame(data = data)
pab_df

,Draw 0,Draw 1,Draw 2,Draw 3,Draw 4,Draw 5,Draw 6,Draw 7,Draw 8,Draw 9,Draw 10
0,0.033447,0.048719,0.064616,0.080351,0.095457,0.109722,0.123118,0.135719,0.147652,0.159054,0.170054


### P(A | B) over a number of draws

In [10]:
palb_df = pab_df/pb_df
palb_df

,Draw 0,Draw 1,Draw 2,Draw 3,Draw 4,Draw 5,Draw 6,Draw 7,Draw 8,Draw 9,Draw 10
0,0.056336,0.069394,0.082023,0.094234,0.106053,0.117518,0.128674,0.139569,0.150253,0.160771,0.171163


### P(B | A) over a number of draws

In [11]:
pbla_df = pab_df/pa_df
pbla_df 

,Draw 0,Draw 1,Draw 2,Draw 3,Draw 4,Draw 5,Draw 6,Draw 7,Draw 8,Draw 9,Draw 10
0,0.473037,0.602893,0.710774,0.79548,0.859111,0.905208,0.937587,0.959726,0.974501,0.984145,0.990313


## Result: Simultaneous Probability Function and Conditional Probability Function for 2 events

In [12]:
def simultaneous_probability_2events(k1,n1,k2,n2,K,N,d):
    return 1-hypergeo2events_cumulative(k1,n1,k2,n2,K+d,N)

def conditional_probability_2events(k1,n1,k2,n2,K,N,d):
    return simultaneous_probability(k1,n1,k2,n2,K,N,d)/hypergeo_cumulative(k1,n1,K+d,N)

## Problem: Probabilities associated with drawing enough lands to cast a card in your opening hand with a given CMC 

In [13]:
# number of cards in the deck
N = 92

# number of lands in hand
l1 = 2

# number of lands in the deck
n = 35

# initial number of cards drawn
K = 7

# CMC of the card, i.e. number of lands drawn
k = 4

# number of lands on bottom of library after mulligan
l2 = 1

In [14]:
data = {f'Draw {i}' : [1-hypergeo_cumulative(k-l1-1,n-l1-l2,i,N)] for i in range(0,11)}
p_df = pd.DataFrame(data = data)
p_df

,Draw 0,Draw 1,Draw 2,Draw 3,Draw 4,Draw 5,Draw 6,Draw 7,Draw 8,Draw 9,Draw 10
0,0.0,0.0,0.11849,0.276477,0.433577,0.571634,0.684697,0.773045,0.839739,0.888738,0.923938


## Problem: Probabilities associated with A = drawing drawing at least one of a card with a given CMC in a game and B = drawing enough basic lands and talismans to cast the card

In [15]:
# number of cards in the deck
N = 99

# number of the given card in the deck
n1 = 1

# number of mana producing lands in the deck
n2 = 40

# number of rampant growths
n3 = 1

# initial number of cards drawn
K = 7

# CMC of the card, i.e. amount of mana the manabase needs to produce
k = 3

### P(A) over a number of draws

In [16]:
data = {f'Draw {i}' : [1-hypergeo_cumulative(0,n1,K+i,N)] for i in range(0,11)}
pa_df2 = pd.DataFrame(data = data)
pa_df2

,Draw 0,Draw 1,Draw 2,Draw 3,Draw 4,Draw 5,Draw 6,Draw 7,Draw 8,Draw 9,Draw 10
0,0.070707,0.080808,0.090909,0.10101,0.111111,0.121212,0.131313,0.141414,0.151515,0.161616,0.171717


### P(B) over a number of draws

In [17]:
def compute_possible_draws(k,ns,K,N):
    possible_draws = {}
    for j in range(k,K+1):
        draws = [0 for _ in ns]
        draws = distribute_k_over_n(j, draws)
        for draw in draws:
            possible_draws[str(draw)] = draw
    
    return [list(zip(ns,list(l))) for _, l in possible_draws.items()]
    #return hypergeo_cumulative(k2-1,n2,K+i,N) # need to employ some sort of integer partitioner

def talisman_prob(k, K, n2, n3, N):    
    possible_draws = compute_possible_draws(k,[n2,n3],K,N)

    total = 0

    for draw in possible_draws:
        k2 = draw[0][1]
        k3 = draw[1][1]

        p = hypergeo2events(k2,n2,k3,n3,K,N)
        
        m = 1 if k2 >= 2 else 0
        
        total += m * p
        
    return total

In [18]:
data = {f'Draw {i}' : [talisman_prob(k, K + i, n2, n3, N)] for i in range(0,11)}
pb_df2 = pd.DataFrame(data = data)
pb_df2

,Draw 0,Draw 1,Draw 2,Draw 3,Draw 4,Draw 5,Draw 6,Draw 7,Draw 8,Draw 9,Draw 10
0,0.615941,0.722932,0.806017,0.867737,0.911947,0.942648,0.963398,0.977085,0.985914,0.991493,0.99495


### P(A and B) over a number of draws

In [19]:
def talisman_prob(k, K, n1, n2, n3, N, debug=False):    
    possible_draws = compute_possible_draws(k,[n2,n3],K,N)

    total = 0

    for draw in possible_draws:
        for k1 in range(1, n1 + 1):
            k2 = draw[0][1]
            k3 = draw[1][1]
            ks = [k1, k2, k3]
            ns = [n1, n2, n3]

            p = hypergeoNevents(ks,ns,K,N)
            
            m = 1 if k2 >= 2 else 0

            total += m * p
            
            if debug:
                print(draw, p, total)
        
    return total

In [21]:
data = {f'Draw {i}' : [talisman_prob(k, K + i, n1, n2, n3, N)] for i in range(0,11)}
pab_df2 = pd.DataFrame(data = data)
pab_df2

,Draw 0,Draw 1,Draw 2,Draw 3,Draw 4,Draw 5,Draw 6,Draw 7,Draw 8,Draw 9,Draw 10
0,0.03498,0.050518,0.066502,0.082168,0.097093,0.111116,0.124252,0.136605,0.14832,0.159542,0.1704


### P(A | B) over a number of draws

In [22]:
palb_df2 = pab_df2/pb_df2
palb_df2

,Draw 0,Draw 1,Draw 2,Draw 3,Draw 4,Draw 5,Draw 6,Draw 7,Draw 8,Draw 9,Draw 10
0,0.056791,0.06988,0.082508,0.094693,0.106468,0.117877,0.128973,0.139809,0.150439,0.160911,0.171265


### P(B | A) over a number of draws

In [23]:
pbla_df2 = pab_df2/pa_df2
pbla_df2

,Draw 0,Draw 1,Draw 2,Draw 3,Draw 4,Draw 5,Draw 6,Draw 7,Draw 8,Draw 9,Draw 10
0,0.494719,0.625164,0.731527,0.813466,0.873835,0.916711,0.946226,0.965994,0.978912,0.987165,0.992328


## Scratchwork

In [23]:
from utils import distribute_k_over_n

k = 3
n1 = 2
n2 = 40
n3 = 1
bl = 1
K = 7
N = 99

def compute_possible_draws(k,ns,K,N):
    possible_draws = {}
    for j in range(k,K+1):
        draws = [0 for _ in ns]
        draws = distribute_k_over_n(j, draws)
        for draw in draws:
            possible_draws[str(draw)] = draw
    
    return [list(zip(ns,list(l))) for _, l in possible_draws.items()]

possible_draws = compute_possible_draws(k, [n2-bl, n3, bl], K, N)

In [24]:
total = 0
for draw in possible_draws:
    for k1 in range(1, n1 + 1):
        k2 = draw[0][1]
        k3 = draw[1][1]
        kl = draw[2][1]
        ks = [k1, k2, k3, kl]
        ns = [n1, n2-bl, n3, bl]
        p = hypergeoNevents(ks,ns,K,N)
        m = 1 if k2 + kl >= 2 and kl < bl else 0
        total += m * p
        print(draw, total)
print(total)

[(39, 3), (1, 0), (1, 0)] 0.03403406236511969
[(39, 3), (1, 0), (1, 0)] 0.03497945298637301
[(39, 2), (1, 1), (1, 0)] 0.03773897155651785
[(39, 2), (1, 1), (1, 0)] 0.037815624850132984
[(39, 2), (1, 0), (1, 1)] 0.037815624850132984
[(39, 2), (1, 0), (1, 1)] 0.037815624850132984
[(39, 1), (1, 2), (1, 0)] 0.037815624850132984
[(39, 1), (1, 2), (1, 0)] 0.037815624850132984
[(39, 1), (1, 1), (1, 1)] 0.037815624850132984
[(39, 1), (1, 1), (1, 1)] 0.037815624850132984
[(39, 1), (1, 0), (1, 2)] 0.037815624850132984
[(39, 1), (1, 0), (1, 2)] 0.037815624850132984
[(39, 0), (1, 3), (1, 0)] 0.037815624850132984
[(39, 0), (1, 3), (1, 0)] 0.037815624850132984
[(39, 0), (1, 2), (1, 1)] 0.037815624850132984
[(39, 0), (1, 2), (1, 1)] 0.037815624850132984
[(39, 0), (1, 1), (1, 2)] 0.037815624850132984
[(39, 0), (1, 1), (1, 2)] 0.037815624850132984
[(39, 0), (1, 0), (1, 3)] 0.037815624850132984
[(39, 0), (1, 0), (1, 3)] 0.037815624850132984
[(39, 4), (1, 0), (1, 0)] 0.05483265603269283
[(39, 4), (1, 0),